In [5]:
import pandas as pd
import re
import nltk
import joblib
import mlflow
import mlflow.sklearn

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

ModuleNotFoundError: No module named 'pandas'

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

NameError: name 'nltk' is not defined

In [ ]:
df = pd.read_csv(rf"D:\Desktop\Data_Science_Intership_\MLflow for Experiment Tracking & Model Management\data.csv")


In [ ]:
df = df[['Review text', 'Ratings']].dropna()
df = df[df['Ratings'] != 3]

df['sentiment'] = df['Ratings'].apply(lambda x: 1 if x >= 4 else 0)
df.rename(columns={'Review text': 'review'}, inplace=True)

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return ' '.join(words)

df['clean_review'] = df['review'].apply(clean_text)

NameError: name 'stopwords' is not defined

In [ ]:
X = df['clean_review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
mlflow.set_experiment("Sentiment_Analysis_Flipkart_YONEX")

2026/02/01 18:42:22 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/01 18:42:22 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location=('file:D:/Desktop/Data_Science_Intership_/MLflow for Experiment Tracking & '
 'Model Management/mlruns/1'), creation_time=1769948841513, experiment_id='1', last_update_time=1769948841513, lifecycle_stage='active', name='Sentiment_Analysis_Flipkart_YONEX', tags={}>

In [ ]:
with mlflow.start_run(run_name="TFIDF_LogReg_v1"):

    # Parameters
    max_features = 5000
    ngram_range = (1, 2)
    C = 1.0

    # Log parameters
    mlflow.log_param("max_features", max_features)
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("C", C)

    # Vectorization
    tfidf = TfidfVectorizer(
        max_features=max_features,
        ngram_range=ngram_range
    )

    X_train_tfidf = tfidf.fit_transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    # Model
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train_tfidf, y_train)

    # Predictions
    y_pred = model.predict(X_test_tfidf)

    # Metrics
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("accuracy", acc)

    # Save artifacts
    joblib.dump(model, "sentiment_model.pkl")
    joblib.dump(tfidf, "tfidf_vectorizer.pkl")

    mlflow.log_artifact("sentiment_model.pkl")
    mlflow.log_artifact("tfidf_vectorizer.pkl")

    # Log model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name="Flipkart_Sentiment_Model"
    )

2026/02/01 18:42:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\jakkoju vikasraj\AppData\Roaming\Python\Python313\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Registered model 'Flipkart_Sentiment_Model' already exists. Creating a new version of this model...
Created version '3' of model 'Flipkart_Sentiment_Model'.
